In [1]:
import sys
import os
sys.path.append(os.path.abspath('./../../../..'))
from cell_extractor.CellDetectorBase import CellDetectorBase
from pipeline.lib.annotation_layer import Cell,random_string
import numpy as np
from random import sample
from pipeline.lib.UrlGenerator import UrlGenerator
from cell_extractor.BorderFinder import BorderFinder
from cell_extractor.CellDetector import MultiThresholdDetector
def numpy_to_json(data,color_hex = None,category = 'Round3_Sure'):
    cells = []
    for coord in data:
        cell = Cell(np.array(coord,dtype=float),random_string())
        cell.category = category
        cell.coord[-1]=cell.coord[-1]+0.5
        if hasattr(cell,'description'):
            del cell.description
        cell_json = cell.to_json()
        if color_hex is not None:
            cell_json["props"] = [color_hex]
        cells.append(cell_json)
    return cells

def create_QC_url(animal,sures,unsures,title):
    urlGen = UrlGenerator()
    image_layer = 'precomputed://https://activebrainatlas.ucsd.edu/data/'+animal+'/neuroglancer_data/'
    urlGen.add_stack_image(animal,channel=1)
    urlGen.add_stack_image(animal,channel=2,color='red')
    urlGen.add_stack_image(animal,channel=3,color='green')
    urlGen.add_annotation_layer('Sure',annotations = sures)
    urlGen.add_annotation_layer('Unsure',annotations = unsures)
    return urlGen.add_to_database(title,34)


In [2]:
animal = 'DK41'
finder = BorderFinder(animal)
detector = MultiThresholdDetector(animal,round = 2)
sure = detector.get_sures()
unsure = detector.get_unsures()
sure.rename(columns = {'x' : 'col', 'y' : 'row'}, inplace = True)
unsure.rename(columns = {'x' : 'col', 'y' : 'row'}, inplace = True)
border_cell,sure = finder.find_border_cells(sure)
border_cell,unsure = finder.find_border_cells(unsure)
sure_data_sample = sure[['col','row','section']].sample(500).sort_values('section').to_numpy().tolist()
sure_data = sure[['col','row','section']].sort_values('section').to_numpy().tolist()
unsure_data = unsure[['col','row','section']].sort_values('section').to_numpy().tolist()
sure_cells_sample = numpy_to_json(sure_data_sample,category = 'Round3_Sure')
sure_cells = numpy_to_json(sure_data,category = 'Round3_Sure')
unsure_cells = numpy_to_json(unsure_data,color_hex='#1d66db',category = 'Round3_Unsure')

sc_id = create_QC_url(animal,sure_cells_sample,unsure_cells,'Julian QC3 '+animal)
false_negative_id = create_QC_url(animal,sure_cells,unsure_cells,'Julian false negative '+animal)
print(f'Julian QC url id {animal} : {sc_id}')
print(f'Julian false negative url id {animal} : {false_negative_id}')

sc_id = create_QC_url(animal,sure_cells_sample,unsure_cells,'Marissa QC3 '+animal)
false_negative_id = create_QC_url(animal,sure_cells,unsure_cells,'Marissa false negative '+animal)
print(f'Marissa QC url id {animal} : {sc_id}')
print(f'Marissa false negative url id {animal} : {false_negative_id}')

calculating borders for DK41
image loaded
contours found
borders calculated
Julian QC url id DK41 : 625
Julian false negative url id DK41 : 626
Marissa QC url id DK41 : 627
Marissa false negative url id DK41 : 628


In [5]:
animal = 'DK56'
finder = BorderFinder(animal)
detector = MultiThresholdDetector(animal,round = 2)
detection = detector.load_detections()
sure = detector.get_sures()
unsure = detector.get_unsures()
len(sure),len(unsure)


NoResultFound: No row was found when one was required

In [21]:
sum(detection.name == '2000_unsure')

2752

In [10]:
detection = detector.load_detections()

In [15]:
animal = 'DK52'
# finder = BorderFinder(animal)
sure_len = []
unsure_len = []
for thresh in [2000,2100,2200,2300,2700]:
    detector = CellDetectorBase(animal,round = 2,segmentation_threshold=thresh)
    detection = detector.load_detections()
    sure = detection[detection.predictions==2]
    unsure = detection[detection.predictions==0]
    sure_len.append(len(sure))
    unsure_len.append(len(unsure))
sure_len,unsure_len,sum(sure_len),sum(unsure_len)

([9488, 36, 6, 113, 11], [2137, 6, 5, 19, 13], 9654, 2180)

In [16]:
animal = 'DK52'
finder = BorderFinder(animal)
detector = MultiThresholdDetector(animal,round = 2)
sure = detector.get_sures()
unsure = detector.get_unsures()
len(sure),len(unsure)

(9500, 2145)

In [ ]:
    Julian QC url id : 621
     Julian false negative url id : 622
     Marissa QC url id : 623
     Marissa false negative url id : 624

In [17]:
detection.head()

,animal,section,row,col,label,mean_score,std_score,predictions
0,DK41,347,5364,28747,0.0,-3.753175,1.660697,-2
1,DK41,347,5324,28755,0.0,-4.274146,1.619854,-2
2,DK41,347,5334,28704,0.0,-8.031691,1.333832,-2
3,DK41,347,5337,28674,0.0,-4.631068,0.967737,-2
4,DK41,347,5338,28678,0.0,-8.720782,1.677284,-2
